In [ ]:

#^###############################  Storage_Optimization.ipynb  ####################################
# ^ Author: Sukhendu Sain
# ^ Description: Main Codebase to give estimate on Zoning and Bin Allocation based on Part Categorization
# ^ Data: 03-Mar-2025
#^################################################################################

In [ ]:
# Import Necessary Libraries, Utils, and Config Files
from config import *
from tqdm import tqdm
from pypdf import PdfReader
import pandas as pd
import numpy as np
import math
import openpyxl
import utils

# Data Import and Clean

## Data Processing & Calculation

In [ ]:

# * SAVE the Marged Main Dataset into Excel, To Avoid The Above Steps During Rerun
#df_Main.to_excel('Gparts-MergedMainData.xlsx', index=False) 
# ~8 Secs

In [ ]:
#####   READ  ALREADY SAVED GPARTS-Merged Data
## @ Read FILE:: (Gparts-MergedMainData.xlsx) into Dataframe
df_Main = utils.read_excel(r"C:\Sukhendu\UPWORK-WORK\0_Dondray_Auto\StorageRackOptimizationNew\Codes\Gparts-MergedMainData.xlsx").fillna("")
# ~6 secs

In [ ]:

## @ Read FILE:: (GMPARTS Parts 2 xlsx Files into Dataframe
GMPARTS_FILE_PATH1 = r"C:\Sukhendu\UPWORK-WORK\0_Dondray_Auto\StorageRackOptimizationNew\Data&Files\All_Parts_for_BS__02_06_2025.xlsx"
GMPARTS_FILE_PATH2 = r"C:\Sukhendu\UPWORK-WORK\0_Dondray_Auto\StorageRackOptimizationNew\Data&Files\All_Parts_for_BS-3__02_06_2025.xlsx"

df_GMParts1 = utils.read_excel(GMPARTS_FILE_PATH1)
df_GMParts2 = utils.read_excel(GMPARTS_FILE_PATH2)

df_GMParts = pd.concat([df_GMParts1, df_GMParts2])
df_GMParts.insert(5, 'OH Inventory', 0)
df_GMParts.insert(3, 'Part Category', "")
df_GMParts.insert(15, 'Zoning', "")
df_GMParts.insert(16, 'StorageType', "")
df_GMParts.insert(17, 'SubStorage', "")
df_GMParts.insert(18, 'Bin Type', "")
df_GMParts.insert(19, 'Num. Bin Required', 0)

df_GMParts = df_GMParts.reset_index(drop=True)
df_GMParts["PART DESCRIPTION"] = df_GMParts["PART DESCRIPTION"].fillna("")

if print_df_after_import: utils.print_df(df_GMParts) # Print the Dataframe
# ~2 mins 38 secs

In [ ]:
#df_GMParts.insert(5, 'OH Inventory', 0)
# df_GMParts.insert(15, 'Zoning', "")
# df_GMParts.insert(16, 'StorageType', "")
# df_GMParts.insert(17, 'SubStorage', "")
# df_GMParts.insert(18, 'Bin Type', "")
# df_GMParts.insert(19, 'Num. Bin Required', 0)

In [ ]:
utils.print_df(df_GMParts, 1)

In [ ]:
#utils.print_df(df_GMParts,50000) # Print the Dataframe

#utils.print_df(pd.DataFrame(df_GMParts['PART DESCRIPTION'].unique()))
#pd.DataFrame(df_GMParts['PART DESCRIPTION'].unique()).to_excel('Only-Parts-Desc.xlsx')

In [ ]:
# Define categorization rules
categorizationRulesGM = [
    ['include', 'nut', 'Nuts'],
    ['include', 'washer', 'Washer'],
    ['include', 'bolt', 'Bolt'],
    ['include', 'pin', 'Pin'],
    ['include', 'lever', 'Transmission'],        
    ['include', 'andcable', 'Cable'],
    ['include', 'bearing', 'Bearings'],
    ['include', 'becu', 'Control Modules & ECUs'],
    ['include', 'lwr cont', 'Chassis'],
    ['include', 'upr cont', 'Chassis'],
    ['include', 'bushing', 'Chassis'],
    ['include', 'belt', 'Belts'],
    ['include', 'bolt', 'Bolts'],
    ['include', 'bowdencable', 'Cable'],
    ['include', 'brk', 'Brakes'],
    ['include', 'bumper', 'Body'],
    ['include', 'cable', 'Cable'],
    ['include', 's/str', 'Latch'],
    ['include', 'cabshaft', 'Cab Shafts'],
    ['include', 'pcv', 'PCV'],
    ['include', 'camshaft', 'Cam Shafts'],
    ['include', 'chain', 'Chains'],
    ['include', 'circlip', 'Clips'],
    ['include', 'clamp', 'Clamp'],
    ['include', 'clip', 'Clips'],
    ['include', 'countergear', 'Counter Gear'],
    ['include', 'countershaft', 'Counter Shafts'],
    ['include', 'cover', 'Covers'],
    ['include', 'crankshaft', 'Shafts'],
    ['include', 'crossmember', 'Chassis'],
    ['include', 'crosswire', 'Wiring'],
    ['include', 'daempferfilter', 'Air Filters'],
    ['include', 'defender', 'Body'],
    ['include', 'door', 'Doors'],
    ['include', 'duraseal', 'Seals'],
    ['include', 'eclipse', 'Clips & Retainers'],
    ['include', 'econoseal', 'Seals'],
    ['include', 'ecu', 'Control Modules & ECUs'],
    ['include', 'eyebolt', 'Eyebolt'],
    ['include', 'fan', 'Fan'],
    ['include', 'fastener', 'Fastener'],
    ['include', 'fender', 'Fender'],
    ['include', 'filter', 'Filters'],
    ['include', 'foam', 'Foam'],
    ['include', 'foglamp', 'Fog Lamp'],
    ['include', 'frame', 'Frames'],
    ['include', 'gasket', 'Gaskets'],
    ['include', 'gear', 'Gear'],
    ['include', 'hairspring', 'HAir Spring'],
    ['include', 'handle', 'Handle'],
    ['include', 'hdlamp', 'Head Lamp'],
    ['include', 'headlamp', 'Head Lamp'],
    ['include', 'hinge', 'Hinge'],
    ['include', 'hose', 'Hose'],
    ['include', 'insulator', 'Insulator'],
    ['include', 'intank', 'Intank'],
    ['include', 'jackscrew', 'Jack Screw'],
    ['include', 'lamp', 'Lamp'],
    ['include', 'latch', 'Latches'],
    ['include', 'latching', 'Latches'],
    ['include', 'light', 'Light'],
    ['include', 'mainshaft', 'Main Shaft'],
    ['include', 'module', 'Control Module'],
    ['include', 'motorcraft', 'Motorcraft'],
    ['include', 'motor', 'Motors'],
    ['include', 'motorcycle', 'Motorcycle'],
    ['include', 'motorola', 'Motors'],
    ['include', 'oilseal', 'Oil Seal'],
    ['include', 'outdoor', 'Outdoor'],
    ['include', 'overfender', 'Over Fender'],
    ['include', 'pinion', 'Pinion'],
    ['include', 'pipe', 'Pipes'],
    ['include', 'polyfoam', 'Ploy Foam'],
    ['include', 'propshaft', 'Prop Shaft'],
    ['include', 'pump', 'Pumps'],
    ['include', 'recovery', 'Others'],
    ['include', 'saddleclamp', 'Saddle Clamp'],
    ['include', 'scotseal', 'Scot Seals'],
    ['include', 'screw', 'Screw'],
    ['include', 'seal', 'Seals'],
    ['include', 'secu', 'Security ECU'],
    ['include', 'sensor', 'Sensors'],
    ['include', 'shaft', 'Shafts'],
    ['include', 'speedclip', 'Speed Clips'],
    ['include', 'spring', 'Springs'],
    ['include', 'steering', 'Steering'],
    ['include', 'strgpump', 'Storage Pump'],
    ['include', 'subframe', 'Sub Frames'],
    ['include', 'taillamp', 'Tail Lamp'],
    ['include', 'tarik', 'Fluid Storage'],
    ['include', 'tanks', 'Tanks'],
    ['include', 'unigear', 'Unigear'],
    ['include', 'unsealed', 'Unsealed'],
    ['include', 'valve', 'Valves'],
    ['include', 'axle', 'Axle'],
    ['include', 'cap', 'Cap'],
    ['include', 'decal', 'Hood'],
    ['include', 'hood', 'Hood'],
    ['include2', 'moulding', 'fascia', 'Moulding Fascia'],    
    ['include2', 'moulding', 'hdlp', 'Headlamp Moulding'],
    ['include2', 'moulding', 'body', 'Body Moulding'],    
    ['include2', 'moulding', 's/d', 'Sidedoor Moulding'],    
    ['include2', 'moulding', 'pnl', 'Panel Moulding'],   
    ['include2', 'moulding', 'fndr', 'Fender Moulding'],  
    ['include', 'mldg', 'Moulding'],       
    ['include', 'moulding', 'Moulding'],    
    ['include', 'bumper', 'bumper'],
    ['include', 'abs', 'Abs'],
    ['include', 'brake', 'Brakes'],
    ['include', 'bpr', 'Bumper'],
    ['include', 'a/c', 'AC'],
    ['include', 'vac', 'HVAC'],
    ['include', 'wrg', 'Wiring'],
    ['include', 'harness', 'Harness'],
    ['include', 'trim', 'Trims'],
    ['include', 'tire', 'Tires'],
    ['include', 'battery', 'Battery'],
    ['include', 'strut', 'Suspension'],
    ['include', 'stab', 'Suspension'],
    ['include2', 'shield', 'skid', 'Suspension'],
    ['include2', 'lugg', 'carr', 'Exterior'],
    ['include2', 'eng', 'indicator', 'Instrumentation'],
    ['include2', 'pmp', 'therm', 'Cooling'],
    ['include2', 'cv', 'cd', 'Media'],
    ['include', 'frame', 'Frame'],
    ['include', 'console', 'Console'],
    ['include', 'strg', 'Steering'],
    ['include', 'seat', 'Seats'],
    ['include', 'engine', 'Engine'],
    ['include', 'radiator', 'Radiator'],
    ['include', 'oil', 'Oil'],
    ['include', 'mirror', 'Mirrors'],
    ['include', 'paint', 'Paint'],
    ['include', 'cluster', 'Cluster'],
    ['include', 'wdo', 'Windows'],
    ['include', 'window', 'Windows'],
    ['include', 'radio', 'Radio'],
    ['include', 'generator', 'Generator'],
    ['include', 'spkr', 'Speaker'],
    ['include', 'reinforcement', 'Reinforcement'],
    ['include', 'windshield', 'Windshield'],
    ['include2', 'blade', 'wsw', 'Wiper'],
    ['include', 'wsw', 'Wiper'],
    ['include', 'wiper', 'Wiper'],
    ['include2', 'head', 'cyl', 'Cylinder Head'],
    ['include2', 'booster', 'p/b', 'Booster p/b'],
    ['include2', 'mount', 'eng', 'Mount Eng'],
    ['include2', 'tank', 'fuel', 'Fuel Tank'],
    ['include2', 'spark', 'plug', 'Spark Plug'],
    ['include2', 'transmitter', 'theft', 'Theft Transmitter'],
    ['include2', 'drv', 'shf', 'Drive Shaft'],
    ['include2', 'prop', 'shf', 'Prop Shaft'],
    ['include2', 'carrier', 'diff', 'Carrier Diff'],
    ['include2', 'starter', 'asm', 'Starter Asy'],
    ['include2', 'switch', 'ign', 'Switch Ignition'],
    ['include2', 'connector', 'harn', 'Connector Harn'],
    ['include2', 'panel', 'asm', 'Panel Asy'],
    ['include2', 'label', 'emis', 'Label Emission'],
    ['include2', 'tube', 'pcv', 'Tube Pcv'],
    ['include2', 'rear', 'mir', 'Rear Mirrors'],
    ['include2', 'front', 'mir', 'Front Mirrors'],
    ['include2', 'view', 'mir', 'Mirrors'],    
    ['include2', 'clu', 'press', 'Press Clutch'],
    ['include2', 'clu', 'cylinder', 'Clutch Cylinder'],
    ['include2', 'wheel', 'kit', 'Wheel Kit'],
    ['include', 'stereo', 'Audio'],
    ['include', 'armrest', 'Arm Rest'],
    ['include', 'panel-rf', 'Interior'],
    ['include', 'airbag', 'Airbag'],
    ['include', 'speaker', 'Speaker'],
    ['include2', 'rail', 'si', 'Rail Si'],
    ['include2', 'rail', 'pnl', 'Rail Panel'],
    ['include2', 'cleaner', 'air', 'Air Cleaner'],
    ['include2', 'manifold', 'int', 'Manifold Interior'],
    ['include2', 'thermostat', 'eng', 'Thermostat Engine'],
    ['include2', 'rod', 'conn', 'Connector Rod'],
    ['include2', 'liner', 'w/h', 'Liner Wheel'],
    ['include2', 'pad', 'tr', 'Interior'],
    ['include2', 'shield', 'w/h', 'Wheel Shield'],
    ['include2', 'plate', 'tr', 'Plate Tr'],
    ['include2', 'nut', 'whl', 'Wheels Nut'],
    ['include2', 'shield', 'air', 'Air Shield'],
    ['include2', 'shield', 'sph', 'Sph Shield'],
    ['include2', 'bat', 'cell', 'Battery Cell'],
    ['include', 'sunshade', 'Sunshade'],
    ['include', 'bezel', 'Bezel'],
    ['include', 'starter', 'Starter'],
    ['include', 'throttle', 'Throttle'],
    ['include', 'throtte', 'Throttle'],
    ['include', 'throttle', 'Throttle'],
    ['include2', 'bar', 'tie', 'Bar Tie'],
    ['include2', 'compartment', 'flr', 'Compartment Floor'],
    ['include2', 'panel', 'w/h', 'Wheel Panel'],
    ['include2', 'core', 'htr', 'Core Heater'],
    ['include2', 'outlet', 'air', 'Outlet Air'],
    ['include2', 'yoke', 'trans', 'Yoke Transmission'],
    ['include2', 'yoke', 'shf', 'Yoke Shaft'],
    ['include2', 'mount', 'trans', 'Mount Transmission'],
    ['include2', 'control', 'trns', 'Control Transmission'],
    ['include2', 'switch', 'park', 'Parking Switch'],
    ['include2', 'yoke', 'susp', 'Yoke Suspension'],
    ['include2', 'yoke', 'prop', 'Yoke Propeller'],
    ['include2', 'yoke', 'diff', 'Yoke Differential'],
    ['include2', 'yoke', 'slip', 'Yoke Slip'],
    ['include2', 'housing', 'axl', 'Axle Housing'],
    ['include2', 'ring', 'pstn', 'Ring Piston'],
    ['include2', 'balancer', 'shf', 'Shaft Balancer'],
    ['include2', 'air', 'inl', 'Air Inlet'],
    ['include2', 'deflector', 'wat', 'Water Deflector'],
    ['include2', 'panel', 'body', 'Panel Body'],
    ['include2', 'hub', 'asm', 'Hub Asy'],
    ['include2', 'hub', 'assem', 'Hub Asy'],
    ['include2', 'stud', 'whl', 'Wheel Stud'],
    ['include2', 'converter', 'exh', 'Exhaust Converter'],
    ['include', 'manifold', 'exh', 'Manifold Exhaust'],
    ['include2', 'converter', 'torq', 'Converter Torque'],
    ['include2', 'lock', 'asm', 'Lock Asy'],
    ['include2', 'canister', 'emis', 'Canister Emissions'],
    ['include2', 'tube', 'emis', 'Tube Emissions'],
    ['include2', 'outlet', 'wat', 'Water Outlet'],
    ['include2', 'grille', 'asm', 'Grille Asy'],
    ['include2', 'case', 'asm', 'Case Asy'],
    ['include2', 'indicator', 'trans', 'Transmission Indicator'],
    ['include2', 'support', 'trans', 'Transmission Support'],
    ['include2', 'indicator', 'trns', 'Transmission Indicator'],        
    ['include2', 'clu', 'mas', 'Clutch Master'],
    ['include2', 'air', 'inl', 'Air Inlet'],
    ['include2', 'support', 'rad', 'Radiator Support'],
    ['include2', 'grille', 'rad', 'Radiator Grille'],
    ['include2', 'tank', 'rad', 'Radiator Tank'],
    ['include2', 'injector', 'fuel', 'Fuel Injector'],
    ['include2', 'gate', 'lift', 'Lift Gate'],
    ['include2', 'gate', 'tail', 'Tail Gate'],
    ['include2', 'sender', 'fuel', 'Fuel Sender'],
    ['include2', 'htr', 'cool', 'Cool Heater'],
    ['include2', 'panel', 'otr', 'Panel Other'],
    ['include2', 'key', 'lk', 'Key Lock'],
    ['include2', 'bat', 'tray', 'Battery Tray'],
    ['include2', 'link', 'chk', 'Check Link'],
    ['include2', 'rod', 'lk', 'Rod Lock'],
    ['include2', 'step', 'asm', 'Step Asy'],
    ['include2', 'head', 'speedo', 'Head Speedometer'],
    ['include2', 'shield', 'noise', 'Shield Noise'],
    ['include', 'transmission', 'Transmission'],
    ['include', 'differential', 'Differential'],
    ['include', 'carpet', 'Carpet'],
    ['include', 'connector', 'Connector'],
    ['include', 'voltage', 'Voltage'],
    ['include', 'transmitter', 'Transmitter'],
    ['include', 'eprom', 'Eprom'],
    ['include', 'regulator', 'Regulator'],
    ['include', 'lifter', 'Lifter'],
    ['include', 'manifold', 'Manifold'],
    ['include', 'bracket', 'Bracket'],
    ['include', 'piston', 'Piston'],
    ['include', 'solv', 'Solvent'],
    ['include', 'compressor', 'Compressor'],
    ['include', 'evaporator', 'Evaporator'],
    ['include', 'duct', 'Duct'],
    ['include', 'htr', 'Heater'],
    ['include', 'gusset', 'Gusset'],        
    ['include', 'navn', 'Navigation'],
    ['include', 'muffler', 'Muffler'],
    ['include', 'bulb', 'Bulb'],
    ['include', 'defletor', 'Deflector'],
    ['include', 'switch', 'Switch'],
    ['include', 'charger', 'Charger'],
    ['include', 'rotor', 'Rotor'],
    ['include', 'fuse', 'Fuse'],
    ['include', 'chrg', 'Charger'],
    ['include', 'boot', 'Boot'],
    ['include', 'shim', 'Shim'],
    ['include', 'radiator', 'Radiator'],
    ['include', 'clutch', 'Clutch'],
    ['include', 'mat', 'Mat'],
    ['include', 'display', 'Display'],
    ['include', 'antenna', 'Antenna'],
    ['include', 'relay', 'Relay'],
    ['include', 'camera', 'Camera'],
    ['include', 'amplifier', 'Amplifier'],
    ['include', 'retainer', 'Retainer'],
    ['include', 'manual', 'Manual'],
    ['include', 'gasket', 'Gasket'],
    ['include', 'susp', 'Suspension'],
    ['include', 'spacer', 'Spacer'],
    ['include', 'molding', 'Molding'],
    ['include', 'emblem', 'Emblem'],
    ['include', 'wheel', 'Wheel'],
    ['include', 'pinion', 'Pinion'],
    ['include', 'lock', 'Lock'],
    ['include', 'pedal', 'Pedal'],
    ['include', 'jack', 'Jack'],
    ['include', 'baffle', 'Baffle'],
    ['include', 'cooler', 'Cooler'],
    ['include', 'weatherstrip', 'Weatherstrip'],
    ['include', 'panel', 'Panel'],
    ['include', 'wire', 'Wire'],
    ['include', 'lid', 'Lid'],
    ['include', 'box', 'Box'],
    ['include', 'i/p', 'Instr Panel'],
    ['include', 'cvr', 'Cover'],
    ['include', 'body', 'Body'],
    ['include', 'applique', 'Applique'],
    ['include2', 'sun', 'r', 'Sun Roof'],        
    ['include2', 'shield', 'ht', 'Shield Heater'],        
    ['include2', 'rail', 'asm', 'Rail Asy'],
    ['include2', 'shield', 'sight', 'Shield Sight'],
    ['include2', 'trans', 'fluid', 'Trans Fluid'],
    ['include2', 'converter', 'ctltc', 'Converter Ctltc'],
    ['include2', 'arm', 'vlv', 'Arm Valve'],        
    ['include', 'bolster', 'Bolster'],
    ['include', 'gate', 'Gate'],
    ['include', 'f/tnk', 'Fuel Tank'],
    ['include', 'hoist', 'Hoist'],
    ['include', 'rod', 'Rods'],
    ['include', 'prom', 'Prom'],
    ['include', 'receiver', 'Receiver'],
    ['include', 'transceiver', 'Transceiver'],
    ['include', 'brace', 'Brace'],
    ['include', 'cyl', 'Cylinder'],
    ['include', 'plug', 'Plug'],
    ['include', 'adjust kit', 'Adjustment Kit'],
    ['include', 'stud', 'Stud'],
    ['include', 'housing', 'Housing'],
    ['include', 'resistor', 'Resistor'],
    ['include', 'base', 'Base'],
    ['include', 'lens', 'Lens'],
    ['include', 'carrier', 'Carrier'],
    ['include', 'blower', 'Blower'],
    ['include', 'terminal', 'Terminal'],
    ['include', 'bar', 'Bars'],
    ['include', 'pkg', 'Package'],
    ['include', 'nozzle', 'Nozzle'],
    ['include', 'knob', 'Knob'],
    ['include', 'restraint', 'Restraint'],
    ['include', 'grille', 'Grille'],
    ['include', 'distributor', 'Distributor'],
    ['include', 'pole piece', 'Pole Piece'],
    ['include', 'accumulator', 'Accumulator'],
    ['include', 'cloth', 'Cloth'],
    ['include', 'solenoid', 'Solenoid'],
    ['include', 'drive', 'Drive'],
    ['include', 'contact', 'Contact'],
    ['include', 'member', 'Member'],
    ['include', 'closeout', 'Closeout'],
    ['include', 'hdl ', 'Handle'],
    ['include2', 'hdlp', 'hsg', 'Headlamp Housing'],
    ['include', 'hub', 'Hub'],
    ['include', 'platform', 'Platform'],
    ['include', 'layout', 'Layout'],
    ['include', 'horn', 'Horn'],
    ['include', 'reservoir', 'Reservoir'],
    ['include', 'filler', 'Filler'],
    ['include', 'alternator', 'Alternator'],
    ['include', 'pulley', 'Pulley'],
    ['include', 'rail', 'Rail'],
    ['include', 'power carb kit', 'Power Carb Kit'],
    ['include', 'rectifier', 'Rectifier'],
    ['include', 'pan', 'Pan'],
    ['include', 'extension', 'Extension'],
    ['include', 'player', 'Player'],
    ['include', 'bow', 'Bow'],
    ['include', 'cbl ', 'Cable'],        
    ['include', 'holder', 'Holder'],
    ['include', 'brush', 'Brush'],
    ['include', 'hanger', 'Hanger'],
    ['include', 'actuator', 'Actuators'],
    ['include', 'carton', 'Carton'],
    ['include', 'tire', 'Tire'],
    ['include', 'grommet', 'Grommets'],
    ['include', 'cab', 'Cabin'],
    ['include', 'tube', 'Tubes'],
    ['include', 'refill', 'Refill'],
    ['include', 'tank', 'Tank'],
    ['include', 'pin', 'Pins'],
    ['include', 'radial', 'Radial'],
    ['include', 'breather', 'Breather'],
    ['include', 'label', 'Labels'],
    ['include', 'tubing', 'Tubing'],
    ['include', 'cleaning', 'Cleaner'],
    ['include', 'plt', 'Plate'],
    ['include', 'stiffener', 'Stiffener'],
    ['include', 'coolant', 'Coolant'],
    ['include', 'pushbutton', 'Push Button'],
    ['include', 'protector', 'Protector'],
    ['include', 'tuning kit', 'Tuning Kit'],
    ['include', 'sprocket', 'Sprocket'],
    ['include', 'net', 'Net'],
    ['include', 'breaker', 'Breaker'],
    ['include2', 'check', 's/d', 'Check Side Door'],
    ['include', 'attachment kit', 'Attachment Kit'],
    ['include', 'insert', 'Inserts'],
    ['include', 'installation kit', 'Installation Kit'],
    ['include', 'case', 'Cases'],
    ['include', 'strap', 'Straps'],
    ['include', 'heater', 'Heating'],
    ['include', 'vent', 'Vents'],
    ['include', 'rivet', 'Rivet'],
    ['include', 'leaf', 'Leaf'],
    ['include', 'impeller', 'IMpeller'],
    ['include', 'striker', 'Striker'],
    ['include', 'arm,vlv rkr', 'Rocker Valve Arm'],
    ['include', 'rack', 'Rack'],
    ['include', 'recorder', 'Recorder'],
    ['include', 'pad', 'Pad'],
    ['include', 'joint', 'Joints'],
    ['include', 'sleeve', 'Sleeves'],
    ['include', 'collar', 'Collars'],
    ['include', 'speedo', 'Speedometer'],
    ['include', 'alarm', 'Alarm'],
    ['include', 'key', 'Keys'],
    ['include', 'flap', 'Flap'],
    ['include', 'drag link', 'Drag Link'],
    ['include', 'Adjuster', 'Adjuster'],
    ['include', 'shackle', 'Shackle'],
    ['include', 'axl', 'Axle'],
    ['include', 'container', 'Container'],
    ['include', 'twc converter', 'Twc Converter'],
    ['include', 'finisher', 'Finisher'],
    ['include', 'dryer', 'Dryer'],
    ['include', 'governor', 'Governor'],
    ['include', 'catalog', 'Catalog'],
    ['include', 'glass', 'Glass'],
    ['include', 'compartment', 'Compartment'], 
    ['include', 'grease', 'Grease'],
    ['include', 'fork', 'Fork'],
    ['include', 'cleaner', 'Cleaners'],
    ['include', 'cam', 'CAM'],
    ['include', 'gls', 'Glass'],
    ['include', 'transistor', 'TRansistor'],
    ['include', 'strainer', 'Strainer'],
    ['include', 'sill', 'Sill'],
    ['include', 'speedometer', 'Speedometer'],
    ['include', 'but asm', 'Buttons Asy'],
    ['include', 'flare', 'Flare'],
    ['include', 'carburetor', 'Carburetor'],
    ['include', 'film', 'Films'],
    ['include', 'monitor', 'Monitor'],
    ['include', 'montr', 'Monitor'],        
    ['include', 'bag', 'Bag'],
    ['include', 'rim', 'Rim'],
    ['include', 'timer', 'Timer'],
    ['include', 'silencer', 'Silencer'],
    ['include', 'tool kit', 'Tool Kit'],
    ['include', 'circuit board', 'Circuit Board'],
    ['include', 'sunshade', 'Sunshade'],
    ['include2', 'shade', 'lugg', 'Luggage Shade'],
    ['include', 'wedge', 'Wedge'],
    ['include', 'calibrator', 'Calibrator'],
    ['include', 'resonator', 'Resonator'],
    ['include', 'divider', 'Divider'],
    ['include', 'core,rad', 'Radiator Core'],
    ['include2', 'guide,vlv', 'lftr', 'Guide Valve Lifter'],
    ['include', 'guide,vlv', 'Guide Valve'],
    ['include2', 'o ring', 'seal-fuel', 'O Ring Seal Fuel'],
    ['include2', 'o ring', 'a/c', 'O Ring Seal A/C'],
    ['include2', 'o ring', 'trans', 'O Ring Seal Transmission'],
    ['include2', 'o-ring', 'drv', 'O Ring Seal Drive'],
 #   ['include', 'pocket', 'Pocket'],    GEAR SET,DIFF RING & DRV
    ['include', 'safety kit', 'Safety Kit'],
    ['include', 'ring,input shf rr brg thr', 'Input Shaft Ring Bearing'],
    ['include', 'ballast kit,hdlp', 'Ballast Kit Headlamp'],
    ['include2', 'gear set', 'diff ring', 'Ring Gear Diff Set'],
    ['include', 'tray', 'Tray'],
    ['include', 'printed circuit', 'Printed Circuit'],
    ['include', 'guard', 'Guard'],
    ['include', 'synchronizer', 'SYnchronizer'],
    ['include', 'weight', 'Weight'],
    ['include', 'controller', 'Controller'],
    ['include', 'adapter', 'Adapters'],
    ['include2', 'pawl', 's/d', 'Side Door Pawl'],
    ['include2', 'pawl', 'gate', 'Gate Pawl'],    
    ['include2', 'pawl', 'seat', 'Seat Pawl'],
    ['include2', 'pawl', 'bumper', 'Bumper Pawl'],
    ['include2', 'pawl', 'bolt', 'Bolt Pawl'],
    ['include', 'nut,s/knu', 'Steer Knuckke Nut'],
    ['include2', 'luggage', 'carrier', 'Luggage Carrier'],
    ['include2', 'gage', 'oil', 'Oil Gauge'],
    ['include2', 'gage', 'fuel', 'Fuel Gauge'],
    ['include2', 'gage', 'temp', 'Temperature Gauge'],
    ['include2', 'gage', 'combo', 'Combo Gauge'],
    ['include2', 'gage', 'boos', 'Boost Gauge'],
    ['include', 'hook', 'Hooks'],
    ['include2', 'telltale', 'info ctr', 'Info Center Telltale'],
    ['include2', 'telltale', 'inst cstr', 'Instrument Cluster Telltale'],
    ['include2', 'telltale', 'instr', 'Instrument Telltale'],
    ['include', 'pillar', 'Pillar'],
    ['include', 'mount', 'Mounts'],
    ['include', 'canister', 'Canister'],
    ['include', 'deadener', 'Deadener Material'],
    ['include', 'cushion', 'Cushion'],
    ['include3', 'stripe', 'flare', 'flame', 'Flare Stripe Flame'],
    ['include2', 'stripe', 'r/cmpt', 'Rear Comp Stripe'],
    ['include3', 'stripe', 'hood', 'ctr', 'Center Hood Stripe'],
    ['include2', 'stripe', 'body', 'Body Stripe'],
    ['include3', 'stripe', 'flr', 'cnsl', 'Floor Console Stripe'],
    ['include2', 'loop', 'clamp', 'Clamp Loop'],
    ['include3', 'loop', 'child', 'Child Seat Loop'],
    ['include3', 'loop', 'plate', 'cargo', 'Plate Cargo Loop'],
    ['include2', 'loop', 'cargo', 'Cargo Loop'],
    ['include2', 'loop', 'belt guide', 'Belt Guide Loop'],
    ['include', 'loop', 'Loop'],
    ['include2', 'card', 'Navn data', 'Electronics'],
    ['include2', 'card', 'field', 'Field Comm Card'],
    ['include2', 'card', 'business', 'Business Card'],
    ['include2', 'card', 'toll', 'Toll Card Holder'],
    ['include2', 'card', 'reader', 'Card Reader'],
    ['include3', 'screen', 'hood', 'air', 'Hood Air Screen'],
    ['include3', 'screen', 'air', 'inl', 'Air Inlet Screen'],
    ['include2', 'screen', 'winterfront', 'Winterfront Screen'],
    ['include2', 'screen', 'grl', 'Grille Screen'],
    ['include2', 'screen', 'spkr', 'Speaker Screen'],
    ['include2', 'flasher', 'emer', 'Emergency Flasher'],
    ['include3', 'flasher', 'hazard', 'sig', 'Hazard Signal Flasher'],
    ['include2', 'flasher', 'hazard', 'Hazard Flasher'],
    ['include', 'flasher', 'Flasher'],
#    ['include', 'cup', 'Interior'],
#    ['include', 'fdtm', 'Fdtm'],
    ['include2', 'slat', 'lugg car', 'Luggage Slat'],
    ['include2', 'slat', '', 'Interior'],
    ['include', 'inslator', 'Insulator'],
    ['include', 'receptacle', 'Receptacle'],
    ['include2', 'link', 'adj', 'Adjustment Link'],
    ['include2', 'link', 'tie', 'Link Tie'],
    ['include2', 'link', 'bolt', 'Bolt Link'],
    ['include2', 'link', 'bracket', 'Bracket Link'],
    ['include2', 'link', 'bushing', 'Bushing Link'],
    ['include', 'link', 'Link'],
    ['include', 'stator', 'Stator'],
    ['include', 'decor strip', 'Decoration Strip'],
    ['include2', 'absorber', 'shk', 'Shock Absorber'],
    ['include2', 'link', 'rod,strg', 'Steering Rod Linkage'],
    ['include2', 'horn', 'asm', 'Horn Asy'],
    ['include2', 'seal kit', 'pinion', 'Pinionn Seal Kit'],
    ['include2', 'retainer', 'pad', 'Retainer Pad'],
    ['include2', 'horn', 'asm', 'Horn Asy'],
    ['include', 'cap,', 'Cap'],
    ['include', 'cap ', 'Cap'],
    ['include', 'washer', 'Washer'],
    ['include', 'printed circuit', 'Electronics'],
    ['include', 'pnl-qtr otr', 'Outer Panel Qtr'],
    ['include2', 'deflector', 'air', 'Air Deflector'],
    ['include2', 'plate', 'sw', 'Interior'],
    ['include2', 'indicator', 'trns', 'Transmission Indicator'],
    ['include2', 'plate', 'clu', 'Plate Clutch'],
    ['include', 'footrest', 'Foot Rest'],
    ['include', 'microphone', 'Microphone'],
    ['include', 'control', 'Controls'],
    ['include', 'escutcheon', 'Escutcheon'],
    ['include', 'p & a price list (comm)', 'Price List (comm)'],
    ['include', 'ballast', 'Ballast'],
    ['include', 'repair kit', 'Repair Kit'],
    ['include', 'cradle', 'Cradle'],
    ['include', 'headphone', 'Head phone'],
    ['include', 'cage', 'Cage'],
    ['include', 'disc', 'Disc'],
    ['include', 'ball', 'Ball'],
    ['include', 'nut-push in', 'Nut Push In'],
    ['include', 'ornament', 'Ornament'],
    ['include', 'fluid', 'Fluid'],
    ['include', 'float kit,carb', 'Float Kit Carb'],
    ['include2', 'plate', 'drvn', 'Plate Drive'],        
    ['include2', 'arm', 'pit', 'Arm Pit'],    
    ['include2', 'wrench', 'whl', 'Wheel Wrench'],   
    ['include2', 'ring', 'ret', 'Ring Retainer'],   
    ['include2', 'diff', 'drv', 'Differential Drive'],  
    ['include2', 'shield', 'eng', 'Shield Engine'],  
    ['include2', 'support', 'sh met', 'Support Sh Metal'],  
    ['include2', 'servo', 'asm', 'Servo Asy'], 
    ['include2', 'lk', 'asm', 'Lock Asy'], 
    ['include2', 'rad', 'drn', 'Radiator Drain'],  
    ['include2', 'mir', 'asm', 'Mirror Asy'], 
    ['include2', 'flange', 'clu', 'Flange Clutch'],          
    ['include2', 'inlet', 'wat', 'Water Inlet'],     
    ['include2', 'pivot', 'hdl', 'Pivot Handle'],     
    ['include2', 'meter', 'fuel', 'Fuel Meter'], 
    ['include2', 'cool', 'air', 'Air Cooler'],     
    ['include2', 'evap', 'core', 'Evaporator Core'],    
    ['include2', 'knock', 'sen', 'Knock Sensor'],     
    ['include2', 'conversion', 'vlv', 'Conversion Valve'],  
    ['include2', 'diff', 'brg', 'Differential Bearing'],      
    ['include2', 'booster', 'audio', 'Booster Audio'],  
    ['include2', 'service', 'audio', 'Service Audio'],            
    ['include', 'detent', 'Detent'],     
    ['include', 'lining', 'Lining'],        
    ['include', ' rad', 'Radiator'],      
    ['include', 'fascia', 'Fascia'],    
    ['include', 'circuit', 'Circuit'],   
    ['include', 'diode', 'Diode'],                                               
    ['include', 'lubricant', 'Lubricant'],  
    ['include', 'harn asm', 'Harness Asy'],
    ['include', 'plate', 'Plate'],
    ['include', 'condenser', 'Condenser'],
    ['include', 'gauge', 'Gauge'],
    ['include', 'thermostat', 'Thermostat'],
    ['include', 'overhaul', 'Overhaul'],
    ['include', 'coupling', 'Coupling'],
    ['include', 'plate', 'Plate'],
    ['include', 'socket', 'Socket'],
    ['include', 'shft', 'Shaft'],
    ['include', 'hdlp', 'Head Lamp'],
    ['include', 'sensing', 'Sensors'],
    ['include', 'buckle', 'Buckle'],
    ['include', 'flange', 'Flange'],
    ['include', 'hge', 'Hinge'],        
    ['include', 'drum', 'Drum'],
    ['include', 'tow', 'Towing'],
    ['include', 'wrench', 'Wrench'],
    ['include', 'channel', 'Channel'],
    ['include', 'calibration', 'calibration'],
    ['include', 'a/rst', 'Armrest'],
    ['include', 'damper', 'Damper'],
    ['include', 'converter', 'Converter'],        
    ['include', 'shft', 'Shaft'],   
    ['include', 'rotator', 'Rotator'],   
    ['include', 'tuning', 'Tuning Kit'],  
    ['include', 'jumper', 'Jumper'],         
    ['include', 'panel asm', 'Panel Asy'],        
    ['include', 'pnl asm', 'Panel Asy'],
    ['include', 'support', 'Support'],
    ['include', 'clu ', 'Clutch'],
    ['include', 'parking', 'Parking'],    
    ['include', 'roller', 'Roller'],     
    ['include', 'balancer', 'Balancer'], 
    ['include', 'tachometer', 'Tachometer'], 
    ['include', 'deflector', 'Deflector'],  
    ['include', 'reflector', 'Reflector'],    
    ['include', 'pivot', 'Pivot'], 
    ['include', 'tape', 'Tape'],         
    ['include', 'ring', 'Ring'],  
    ['include', 'marketing', 'Marketing'],  
    ['include', 'pnl', 'Panel'],   
    ['include', 'dampener', 'Dampener'],  
    ['include', 'odometer', 'Speedometer'],  
    ['include', 'adhesive', 'Tape'],  
    ['include', 'shutter', 'Shutter'],  
    ['include', 'knuckle', 'Knuckle'],  
    ['include', 'shield', 'Shield'],  
    ['include', 'decor', 'Decor'],          
    ['include', 'nipple', 'Nipple'], 
    ['include', 'exh ', 'Exhaust'], 
    ['include', 'step', 'Exterior'],
    ['include', 'liner', 'Engine'],
    ['include', 'scoop', 'Body']
]


In [ ]:

## ^ Part Categorization  - OLD  BUT Client may Share Actual Part Category in Future       
def part_categorization(df_toBeCategorized, categoryColName):
    # TODO: Add more categories
    for i in range(df_toBeCategorized.shape[0]):
        desc = "-".join(df_toBeCategorized.loc[i, "Part Desc."].split("-")[1:])
        pnum = df_toBeCategorized.loc[i, "Part#"][1:].upper()
        category = ""
        if "battery" in desc.lower():
            if desc.strip().lower() == "battery":
                category = "Battery"
            else:
                category = "Battery Accessory"
        elif "tire" in desc.lower():
            if desc.strip().lower() == "tire":
                category = "Tire"
            else:
                category = "Tire Accessory"            
        elif "hood" in desc.lower():
            if (desc.strip().lower() == "hood asy") | (desc.strip().lower() == "hood  asy"):
                category = "Hood"
            else:
                category = "Hood Accessory"
                
        elif (desc.strip().lower() == "cover"):
            category = "Bumper Cover"
        elif ("17D957" in pnum) | ("17K835" in pnum):
            category = "Bumper Cover"
        elif desc.strip().lower() == "seal":
            category = "Seal"
        elif desc.strip().lower() == "name plate":
            category = "Name Plate"
        elif desc.strip().lower() == "v-belt":
            category = "V-Belt"
        elif desc.strip().lower() == "v-belt":
            category = "V-Belt"
       
        elif ("blade" in desc.lower()) & ("wiper" in desc.lower()):
            category = "Wiper Blade"        
        elif ("arm" in desc.lower()) & ("wiper" in desc.lower()):
            category = "Wiper Arm"
        elif ("belt" in desc.lower()) & ("retractor" not in desc.lower()) & ("hole" not in desc.lower()) & ("cover" not in desc.lower()):
            category = "Belt"
        # elif ("hose" in desc.lower()) & ("vent" not in desc.lower()) & ("connect" not in desc.lower() & ("radiator" not in desc.lower()& ("heater" not in desc.lower()):
        #     category = "Hose"

        df_toBeCategorized.loc[i, categoryColName] = category
        

In [ ]:
def part_categorization2(df_toBeCategorized, categorizationRules, categoryColName, categoryDescName):
    
    # Apply categorization
    def categorize(desc):
        desc = "-".join(desc.split("-")[1:]).lower()
        category = "Miscellaneous"
        
        for cat in categorizationRules:
            if cat[0] == 'equal' and cat[1].lower() == desc:
                category = cat[2]
                break
            elif cat[0] == 'include' and cat[1].lower() in desc:
                category = cat[2]
                break
            elif cat[0] == 'include2' and cat[1].lower() in desc and cat[2].lower() in desc:
                category = cat[3]
                break
        
        return category
        
    df_toBeCategorized[categoryColName] = df_toBeCategorized[categoryDescName].astype(str).apply(categorize)

    

In [ ]:
def part_categorizationGM(df_toBeCategorized, categorizationRules, categoryColName, categoryDescName):
    
    # Apply categorization
    def categorizeGM(desc):
        desc = desc.lower().strip()       
        category = "Miscellaneous"
        
        for cat in categorizationRules:
            if cat[0] == 'equal' and cat[1] == desc:
                category = cat[2]
                break
            elif cat[0] == 'include' and cat[1] in desc:
                category = cat[2]
                break
            elif cat[0] == 'include2' and cat[1] in desc and cat[2] in desc:
                category = cat[3]
                break
            elif cat[0] == 'include3' and cat[1] in desc and cat[2] in desc and cat[3] in desc:
                category = cat[4]
                break
        
        return category
        
    df_toBeCategorized[categoryColName] = df_toBeCategorized[categoryDescName].astype(str).apply(categorizeGM)

    

In [ ]:
part_categorization2(df_Main, categorizationRulesGM, 'Part Category 2', 'Part Desc.')
# ~8 secs

In [ ]:
part_categorizationGM(df_GMParts, categorizationRulesGM, 'Part Category', 'PART DESCRIPTION')

In [ ]:
df_GMParts["Part Category"].unique().shape[0], utils.print_df(pd.DataFrame(df_GMParts["Part Category"].value_counts()))

In [ ]:
#utils.print_df(df_Main)

In [ ]:
utils.print_df(pd.DataFrame(df_GMParts[df_GMParts["Part Category"] == "Miscellaneous"]['PART DESCRIPTION'].value_counts()), style='plain')

In [ ]:
df_GMParts[df_GMParts["Part Category"] != "Miscellaneous"].shape[0]

In [ ]:
df_Main[df_Main["Part Category 2"] != "Miscellaneous"].shape[0], df_Main[df_Main["Part Category 2"] == "Miscellaneous"].shape[0]

In [ ]:
df_GMParts[:1048500]

In [ ]:
df_GMParts[:100]

### Apply Zoning

In [ ]:

# @ Apply Zoning based on Time Period/Sale

## * Main Function for Apply Zoning
def Apply_Zoning(df_toBeZoned, zones, soldColName='Total Sold', zoneColName='Zone'): 
    df_toBeZoned.loc[:, zoneColName] = df_toBeZoned[soldColName].apply(lambda x: next((zone for zone, ratio in zones.items() if x >= ratio), list(zones.keys())[-1]))
    df_toBeZoned.loc[df_toBeZoned[soldColName] < 0, zoneColName] = None

In [ ]:

## * Run the Apply_Zoning on df_Main
Apply_Zoning(df_Main, zones, 'Total Sold', 'Zone')

In [ ]:
## * Check each Zone's number of Part Numbers
df_Main['Zone'].value_counts()

In [ ]:
####   TRANSFER  ZONING from  OlD GPARTS (df_Main)  to  df_GMParts  based  on pART cATEGORY

df_Main[:1]

In [ ]:
import itertools 

def get_unique_combinations(df, cols):
    # Convert selected columns to Series
    series_list = [df[col] for col in cols]
    
    # Get unique combinations
    unique_combos = list(itertools.combinations(pd.Series([s.unique() for s in series_list]), len(cols)))
    
    return unique_combos

get_unique_combinations(df_Main[['Part Category 2', 'Zone']], ['Part Category 2', 'Zone'])

In [ ]:
df_Main[['Part Desc.', 'Part Category 2', 'Zone']]

In [ ]:
stris = []

for i in range(df_GMParts.shape[0]):
    stris.append(str(df_GMParts.loc[i, 'PART DESCRIPTION']) + ' | ' +  str(df_GMParts.loc[i, 'Part Category']) + ' | ' + str(df_GMParts.loc[i, 'Zoning']))

utils.print_df(pd.DataFrame(pd.Series(stris).unique()).sort_values(by=0, axis=0, ascending=True))

In [ ]:
stris = []

for i in range(df_Main.shape[0]):
    stris.append(str(df_Main.loc[i, 'Part Desc.']) + ' | ' +  str(df_Main.loc[i, 'Part Category 2']) + ' | ' + str(df_Main.loc[i, 'Zone']))

utils.print_df(pd.DataFrame(pd.Series(stris).unique()).sort_values(by=0, axis=0, ascending=True))

In [ ]:

df_GMParts[:10]

### Specialty Storage Assignment

#### Function for Bin Calculation

In [ ]:
def getNumOfBin(depth, width, height, raw_bin_dim, ohInven, fillFactor):
    # * Raw Bin Dimensions has this format :-  Height_Depth_Width
    if (raw_bin_dim != "") and (ohInven > 0):
        bin_height = float(raw_bin_dim.split("_")[1])
        bin_depth = float(raw_bin_dim.split("_")[2])
        bin_width = float(raw_bin_dim.split("_")[3])

        if raw_bin_dim.split("_")[0] == "BR":   # * Battery Rack
            return round((ohInven * width) / bin_depth, 3)      
               
        volBin = fillFactor * bin_height * bin_depth * bin_width    # * Available Storage Space
        volPart = height * depth * width
        if (volBin == 0):
            return 0
        
        numOfBins = round((ohInven * volPart) / volBin, 3)      # * Returns Fraction. 
        return numOfBins
    else:
        return 0

### Main Function for Storage Assignment

In [ ]:
def getStorage(zone, pcate, depth, width, height, ohInven, fillFactor):
    # Initialize the empty Variables
    storageType = ""
    subStorage = ""

    if zone == "":
        return storageType, subStorage, "", 0 # Return the Values 

    isSpec, storageType, subStorage, raw_bin_dim = utils.getSpecialtyStorage(pcate, depth, width, height)

    if not isSpec: 
        if (zone == "Red Hot") | (zone == "Red"):
            storageType, subStorage, raw_bin_dim = utils.getRedHotStorage(depth, width, height)
        elif (zone == "Orange") | (zone == "Yellow"):
            storageType, subStorage, raw_bin_dim = utils.getOrangeYellowStorage(depth, width, height)
        elif (zone == "Green") | (zone == "Blue"):   
            storageType, subStorage, raw_bin_dim = utils.getGreenBlueStorage(depth, width, height)
 
    numOfBins = getNumOfBin(depth, width, height, raw_bin_dim, ohInven, fillFactor)
    binDim = ""

    # * Build Bin Label with C (Clip), B (Bulk), D (Drawer), Battery Rack (BR), Tire Rack (TR) and Width-Depth-Height
    if raw_bin_dim.strip():   
        binDim =  raw_bin_dim.split('_')[0] + raw_bin_dim.split('_')[3] + raw_bin_dim.split('_')[2] + raw_bin_dim.split('_')[1]
    

    return storageType, subStorage, binDim, numOfBins # Return the Values

#### Apply the Storage Function

In [ ]:
for i in tqdm(range(df_Main.shape[0]), desc="Completion"):
#for i in range(df_Main.shape[0]):
    # Set the Dimensions of the Data into Variables
    depth = df_Main.loc[i, "Depth"]
    height = df_Main.loc[i, "Height"]
    width = df_Main.loc[i, "Width"]

    zone = df_Main.loc[i, "Zone"]
    pcate = df_Main.loc[i, "Part Category"]
    ohInven = df_Main.loc[i, "OH Inventory"]

    # * If any dimension is zero, set empty Storage
    if df_Main.loc[i, "0Dimensions"] == True:
        df_Main.loc[i, "StorageType"] = ""
        df_Main.loc[i, "SubStorage"] = ""
        continue

    # Set Storage of the Parts
    df_Main.loc[i, "StorageType"], df_Main.loc[i, "SubStorage"], df_Main.loc[i, "Bin Type"], df_Main.loc[i, "Num. Bin Required"] = getStorage(zone, pcate, depth, width, height, ohInven, fillFactor)

# ~34 secs

In [ ]:

#  WRITE the  df_GMParts   With  Zoning  and  Bin Allocation
df_GMParts[:1048500].to_excel("gmdatacategorize.xlsx")
df_GMParts[1048500:].reset_index(drop=True).to_excel("gmdatacategorize1.xlsx")



In [ ]:

## @ HANGING Storage Calculation
# TODO: To Get the SKU Count for Hanging Storage
# * ASSUMPTION: Hook Length Based on SKU Count:
# *             6-inch hooks: For SKUs with 10 items or fewer
# *             12-inch hooks: For SKUs with 10–20 items

for hangingPN in tqdm(df_Main.loc[df_Main['StorageType'] == 'Hanging Specialty Storage', "Part#"], "Completion"): # Get Hanging Parts
    # & df_Main.loc[(df_Main['Part#'] == hangingPN), "Num. Bin Required"] = round(int(df_Main.loc[(df_Main['Part#'] == hangingPN), "OH Inventory"].values[0]) / hookDiv, 4)
    df_Main.loc[(df_Main['Part#'] == hangingPN), "SubStorage"], df_Main.loc[(df_Main['Part#'] == hangingPN), "Bin Type"], hookDiv = ("6-inch Hook", "HS06", 10) if df_Main.loc[(df_Main['Part#'] == hangingPN), "SKU Count"].values[0] <= 10 else ("12-inch Hook", "HS12", 20)
    df_Main.loc[(df_Main['Part#'] == hangingPN), "Num. Bin Required"] = int(df_Main.loc[(df_Main['Part#'] == hangingPN), "OH Inventory"].values[0])  # * Set No. of Hooks = Inventory Count

###  SET SKU Count to zero for all other STORAGE Types 
df_Main.loc[df_Main['StorageType'] != 'Hanging Specialty Storage', "SKU Count"] = 0

# ~46 secs

In [ ]:

## @ Tire Storage Calculation
# TODO: FINALIZE  the Calculation of Tire Carousel Model Selection Based on Percentage 
# * ASSUMPTION:  Assign Carousel Model based on Diameter Group %age
# *   If 50% of Tires Have 33-inch or More Diameter, Assign Large-Storage (72-width carousel)
# *   ELSE,  For 28-inch or less, and,  between 28-33 inches, assign standard carrousel (48-width carousel) 
carousel_model = ''
if df_Main[df_Main['StorageType'] == 'Tire Specialty Storage'].shape[0] > 0: 
    carousel_model = 'TR72' if df_Main[df_Main['StorageType'] == 'Tire Specialty Storage'][df_Main['SubStorage'] == '33-inches Dia'].shape[0] / df_Main[df_Main['StorageType'] == 'Tire Specialty Storage'].shape[0] >= tirePercent else 'TR48'
    carousel_width = 72 if df_Main[df_Main['StorageType'] == 'Tire Specialty Storage'][df_Main['SubStorage'] == '33-inches Dia'].shape[0] / df_Main[df_Main['StorageType'] == 'Tire Specialty Storage'].shape[0] >= tirePercent else 48
    for tirePN in df_Main.loc[df_Main['StorageType'] == 'Tire Specialty Storage', "Part#"]:
        df_Main.loc[(df_Main['Part#'] == tirePN), "Num. Bin Required"] = round(int(df_Main.loc[(df_Main['Part#'] == tirePN), "OH Inventory"].values[0]) / (carousel_width // int(df_Main.loc[(df_Main['Part#'] == tirePN), "Width"].values[0])), 3)
        df_Main.loc[(df_Main['Part#'] == tirePN), "Bin Type"] = carousel_model

# ~0.3 secs

In [ ]:

# @ Do Actual Storage Allocation based on Inventory, Number of Bins availiable, Handle Overflow, etc...

df_binData = pd.DataFrame(columns=['Bin Label', 'Bin Category', 'Total Bins', 'Filled Amount', 'Bin Order', 'Bin Location', 'Availiability Flag'])

# * High-Density Drawers (2)
binData = [ 
    {'Bin Label': 'D362406', 'Bin Category': 'Drawer', 'Total Bins': 5, 'Filled Amount': 0, 'Bin Order': 1, 'Bin Location': 'None', 'Availiability Flag': 'Yes'},
    {'Bin Label': 'D482406', 'Bin Category': 'Drawer', 'Total Bins': 4, 'Filled Amount': 0, 'Bin Order': 2, 'Bin Location': 'None', 'Availiability Flag': 'Yes'}
 ]
# * Clip-Shelving (6)
binData.extend([
    {'Bin Label': 'C361215', 'Bin Category': 'Clip', 'Total Bins': 4, 'Filled Amount': 0, 'Bin Order': 3, 'Bin Location': 'None', 'Availiability Flag': 'Yes'},
    {'Bin Label': 'C361815', 'Bin Category': 'Clip', 'Total Bins': 6, 'Filled Amount': 0, 'Bin Order': 4, 'Bin Location': 'None', 'Availiability Flag': 'Yes'},
    {'Bin Label': 'C362415', 'Bin Category': 'Clip', 'Total Bins': 8, 'Filled Amount': 0.0, 'Bin Order': 5, 'Bin Location': 'None', 'Availiability Flag': 'Yes'},    
    {'Bin Label': 'C481215', 'Bin Category': 'Clip', 'Total Bins': 3, 'Filled Amount': 0.0, 'Bin Order': 6, 'Bin Location': 'None', 'Availiability Flag': 'Yes'},  
    {'Bin Label': 'C481815', 'Bin Category': 'Clip', 'Total Bins': 5, 'Filled Amount': 0.0, 'Bin Order': 7, 'Bin Location': 'None', 'Availiability Flag': 'Yes'},   
    {'Bin Label': 'C482415', 'Bin Category': 'Clip', 'Total Bins': 7, 'Filled Amount': 0.0, 'Bin Order': 8, 'Bin Location': 'None', 'Availiability Flag': 'Yes'}
])

# * Bulk-Storage (18)
binData.extend([
  {'Bin Label': 'B482448', 'Bin Category': 'Bulk', 'Total Bins': 4, 'Filled Amount': 0.0, 'Bin Order': 9, 'Bin Location': 'None', 'Availiability Flag': 'Yes'},
  {'Bin Label': 'B483648', 'Bin Category': 'Bulk', 'Total Bins': 5, 'Filled Amount': 0.0, 'Bin Order': 10, 'Bin Location': 'None', 'Availiability Flag': 'Yes'},
  {'Bin Label': 'B484248', 'Bin Category': 'Bulk', 'Total Bins': 6, 'Filled Amount': 0.0, 'Bin Order': 11, 'Bin Location': 'None', 'Availiability Flag': 'Yes'},
  {'Bin Label': 'B484848', 'Bin Category': 'Bulk', 'Total Bins': 4, 'Filled Amount': 0.0, 'Bin Order': 12, 'Bin Location': 'None', 'Availiability Flag': 'Yes'},
  {'Bin Label': 'B487248', 'Bin Category': 'Bulk', 'Total Bins': 3, 'Filled Amount': 0.0, 'Bin Order': 13, 'Bin Location': 'None', 'Availiability Flag': 'Yes'},
  {'Bin Label': 'B489648', 'Bin Category': 'Bulk', 'Total Bins': 4, 'Filled Amount': 0.0, 'Bin Order': 14, 'Bin Location': 'None', 'Availiability Flag': 'Yes'},
  {'Bin Label': 'B722448', 'Bin Category': 'Bulk', 'Total Bins': 4, 'Filled Amount': 0.0, 'Bin Order': 15, 'Bin Location': 'None', 'Availiability Flag': 'Yes'},
  {'Bin Label': 'B723648', 'Bin Category': 'Bulk', 'Total Bins': 3, 'Filled Amount': 0.0, 'Bin Order': 16, 'Bin Location': 'None', 'Availiability Flag': 'Yes'},
  {'Bin Label': 'B724248', 'Bin Category': 'Bulk', 'Total Bins': 4, 'Filled Amount': 0.0, 'Bin Order': 17, 'Bin Location': 'None', 'Availiability Flag': 'Yes'},
  {'Bin Label': 'B724848', 'Bin Category': 'Bulk', 'Total Bins': 6, 'Filled Amount': 0.0, 'Bin Order': 18, 'Bin Location': 'None', 'Availiability Flag': 'Yes'},
  {'Bin Label': 'B727248', 'Bin Category': 'Bulk', 'Total Bins': 4, 'Filled Amount': 0.0, 'Bin Order': 19, 'Bin Location': 'None', 'Availiability Flag': 'Yes'},
  {'Bin Label': 'B729648', 'Bin Category': 'Bulk', 'Total Bins': 5, 'Filled Amount': 0.0, 'Bin Order': 20, 'Bin Location': 'None', 'Availiability Flag': 'Yes'},
  {'Bin Label': 'B962448', 'Bin Category': 'Bulk', 'Total Bins': 4, 'Filled Amount': 0.0, 'Bin Order': 21, 'Bin Location': 'None', 'Availiability Flag': 'Yes'},
  {'Bin Label': 'B963648', 'Bin Category': 'Bulk', 'Total Bins': 5, 'Filled Amount': 0.0, 'Bin Order': 22, 'Bin Location': 'None', 'Availiability Flag': 'Yes'},
  {'Bin Label': 'B964248', 'Bin Category': 'Bulk', 'Total Bins': 7, 'Filled Amount': 0.0, 'Bin Order': 23, 'Bin Location': 'None', 'Availiability Flag': 'Yes'},
  {'Bin Label': 'B964848', 'Bin Category': 'Bulk', 'Total Bins': 4, 'Filled Amount': 0.0, 'Bin Order': 24, 'Bin Location': 'None', 'Availiability Flag': 'Yes'},
  {'Bin Label': 'B967248', 'Bin Category': 'Bulk', 'Total Bins': 6, 'Filled Amount': 0.0, 'Bin Order': 25, 'Bin Location': 'None', 'Availiability Flag': 'Yes'},
  {'Bin Label': 'B969648', 'Bin Category': 'Bulk', 'Total Bins': 4, 'Filled Amount': 0.0, 'Bin Order': 26, 'Bin Location': 'None', 'Availiability Flag': 'Yes'}
])

# * Specialty (7) TR
binData.extend([
    {'Bin Label': 'BR484816', 'Bin Category': 'Battery', 'Total Bins': 14, 'Filled Amount': 0, 'Bin Order': 0, 'Bin Location': 'None', 'Availiability Flag': 'Yes'},
    {'Bin Label': 'TR48', 'Bin Category': 'Tire', 'Total Bins': 6, 'Filled Amount': 0, 'Bin Order': 0, 'Bin Location': 'None', 'Availiability Flag': 'Yes'},
    {'Bin Label': "TR72", 'Bin Category': 'Tire', 'Total Bins': 6, 'Filled Amount': 0, 'Bin Order': 0, 'Bin Location': 'None', 'Availiability Flag': 'Yes'},
    {'Bin Label': 'BC967248', 'Bin Category': 'Bumper Cover', 'Total Bins': 3, 'Filled Amount': 0.0, 'Bin Order': 0, 'Bin Location': 'None', 'Availiability Flag': 'Yes'},  
    {'Bin Label': 'BH967280', 'Bin Category': 'Hood', 'Total Bins': 5, 'Filled Amount': 0.0, 'Bin Order': 0, 'Bin Location': 'None', 'Availiability Flag': 'Yes'},   
    {'Bin Label': 'HS06', 'Bin Category': 'Hanging', 'Total Bins': 5, 'Filled Amount': 0.0, 'Bin Order': 0, 'Bin Location': 'None', 'Availiability Flag': 'Yes'},   
    {'Bin Label': 'HS12', 'Bin Category': 'Hanging', 'Total Bins': 7, 'Filled Amount': 0.0, 'Bin Order': 0, 'Bin Location': 'None', 'Availiability Flag': 'Yes'}
])

# Append the Data to the DF
df_binData = pd.concat([df_binData, pd.DataFrame(binData)], ignore_index=True)
df_binData['Total Bins'] = np.random.choice(range(40, 400), df_binData.shape[0])

In [ ]:
utils.print_df(df_binData)

In [ ]:
# * Write  the Final BINS Dataset into Excel
#df_binData.to_excel('All_Bins_Data.xlsx', index=False) 

In [ ]:

## ^ Psuedocode for main Logic
# ~ Start with 'Bin Type' as default assignment for the part --  ALLOCATE Bin As Per Availiability, Zone, and Priority Rules
# ~ Add a column to 'BINS' dataframe, as Order or priority based on dimension for a given Rack Type (Drawer, Bulk, Clip, etc). 
# ~  This Order should only within specific Rack Type (Drawer, Bulk, Clip, etc). To change to Rack Type, use priorities given for 
# ~ Zones and Part Category. Always start with minimum dimension of the BIN
# ~ Start with 'Red Hot' Zone Parts to allocate Actual Bin. 
# ~ LOOP Over Zones. Then LOOP Over Parts. For Each Part, Check default 'Bin Type'. If Available for storage, then allocate it  
# ~ (Add the Bin Type in 'Actual Bin Allocation',  Increment the Fill_Quantity for the 'Bin Type' in second dataframe for BINS. 
# ~ Fill_Quantity can be fraction.  If Fill_Quantity is same  as  Available Bins, then Set the 'Available FLAG' for the BIN to 'NO'
# ~ That means, the BIN is fully occupied. Pick the next available BIN. 
# ~ Think About Overflow,  and Bin Location (Not sure if Bin Locations will be zone-wise)

In [ ]:
####  BIN  ALLOCATION
## @ Red Hot, Red, Orange, and Yellow Zone Non-Specialty Parts
for pn in tqdm(df_Main.loc[(df_Main['Zone'] == 'Red Hot') | (df_Main['Zone'] == 'Red') | (df_Main['Zone'] == 'Orange') | (df_Main['Zone'] == 'Yellow'), 'Part#'], "Completion"):
    # Get & Set Variables
    actualBin =  ""
    overflowBin = ""
    overflowComment = ""

    partData = df_Main[df_Main['Part#'] == pn]
    partHeight = partData['Height'].values[0]
    partWidth = partData['Width'].values[0]
    partDepth = partData['Depth'].values[0]
    partVolume = partHeight * partWidth * partDepth 
    binType = partData['Bin Type'].values[0]
    storageType = partData['StorageType'].values[0]
    partOHInven = partData['OH Inventory'].values[0]


    # ~ Base Continue Case - If no Storage Assignment, no Inventory Parts or is Specialty Part
    if (binType == "") or ("Specialty Storage" in storageType) or (partOHInven <= 0):     
        continue

    binData = df_binData.loc[df_binData['Bin Label'] == binType]
    totalBinOfType = binData['Total Bins'].values[0]
    filledAmtOfBin = binData['Filled Amount'].values[0]
    binOrder = binData['Bin Order'].values[0]
    flagAvail = binData['Availiability Flag'].values[0]

    # Calculate Variables
    binVolume = fillFactor * (float(binType[1:3]) * float(binType[3:5]) * float(binType[5:7]))
    remainingBinVolume = float(totalBinOfType - filledAmtOfBin) * binVolume  # Check for Remaining Vol in Bin 
    partsAllocated = min(math.floor(remainingBinVolume / partVolume), partOHInven)

    if partsAllocated > 0:   # * If Actual Bin is Availiable
        # Calculate Variables
        totalPartVolume = partOHInven * partVolume
        actualBinRequired = round((partsAllocated * partVolume) / binVolume, 3)
        numBins = round(totalPartVolume / binVolume, 3)

        # Set Values
        df_binData.loc[df_binData['Bin Label'] == binType, 'Filled Amount'] += actualBinRequired
        actualBin = f"{binType} ({actualBinRequired}, {partsAllocated})"

        if filledAmtOfBin + actualBinRequired >= (totalBinOfType - 0.01): # If Bin is FULL, Reset Flag
            df_binData.loc[df_binData['Bin Label'] == binType, 'Availiability Flag'] = 'No'

        if (filledAmtOfBin + numBins) > (totalBinOfType- 0.01):   # * If Overflow (Actual Bin can't fit all Parts) (Only 1)
            overflowParts = partOHInven - partsAllocated
            
            # Find next availiable BIN to  handle overflow parts
            binFound = False
            for binType1 in df_binData.loc[(df_binData['Bin Order'] > binOrder), 'Bin Label']:
                # Calculate Variables
                binData = df_binData[df_binData['Bin Label'] == binType1]
                totalBinOfType = binData['Total Bins'].values[0]
                filledAmtOfBin = binData['Filled Amount'].values[0]
                binVolume = fillFactor * (float(binType1[1:3]) * float(binType1[3:5]) * float(binType1[5:7]))
                remainingBinVolume = float(totalBinOfType - filledAmtOfBin) * binVolume  # Check for Remaining Vol in Bin 
                partsAllocated = min(math.floor(remainingBinVolume / partVolume), partOHInven)

                if partsAllocated > 0:   # If Actual Bin is Availiable
                    binFound = True
                    break
                #  OLD Code
                # if (df_binData[df_binData['Bin Label'] == binType1]['Availiability Flag'].values[0] == 'Yes'):
                #     binData = df_binData[df_binData['Bin Label'] == binType1]
                #     fillAmt = df_binData[df_binData['Bin Label'] == binType1]['Filled Amount'].values[0] 
                #     totalBin = df_binData[df_binData['Bin Label'] == binType1]['Total Bins'].values[0] 
                #     if (fillAmt < (totalBin - 0.01)):
                #         break

            if not (binFound):
                df_Main.loc[df_Main['Part#'] == pn, 'Overflow Bins'] = f"Part1- No More Available/Fitting Bins"
                df_Main.loc[df_Main['Part#'] == pn, 'Overflow Comment'] = f"{overflowParts} Parts Left; No Bins availiable to fit"
                continue

            # Get & Calculate Variables
            binType = binData['Bin Label'].values[0]
            totalBinOfType = binData['Total Bins'].values[0]
            filledAmtOfBin = binData['Filled Amount'].values[0]
            
            binVolume = fillFactor * (float(binType[1:3]) * float(binType[3:5]) * float(binType[5:7]))
            remainingBinVolume = float(totalBinOfType - filledAmtOfBin) * binVolume  # Check for Remaining Vol in Bin 
            partsAllocated = min(math.floor(remainingBinVolume / partVolume), overflowParts)
            actualBinRequired = round((partsAllocated * partVolume) / binVolume, 3)

            # Add  FilledAmount for OverFlow Bin  
            df_binData.loc[df_binData['Bin Label'] == binType, 'Filled Amount'] += actualBinRequired
            overflowBin = f"{binType} ({actualBinRequired}, {partsAllocated})"
        
            # If more overflow
            leftParts = overflowParts - partsAllocated
            if leftParts > 0:
                binsNeeded = round(leftParts * partVolume / binVolume, 3)
                overflowComment = f"Second Overflow: {leftParts} Parts Left; {binsNeeded} quantity of {binType} Bin Needed;"
            
            if filledAmtOfBin + actualBinRequired >= (totalBinOfType - 0.01): 
                df_binData.loc[df_binData['Bin Label'] == binType, 'Availiability Flag'] = 'No'

    else:    # * If Actual Bin is not avaialble
        binFound = False
        for binType1 in df_binData.loc[(df_binData['Bin Order'] > binOrder), 'Bin Label']:
            # Calculate Variables
            binData = df_binData[df_binData['Bin Label'] == binType1]
            totalBinOfType = binData['Total Bins'].values[0]
            filledAmtOfBin = binData['Filled Amount'].values[0]
            binVolume = fillFactor * (float(binType1[1:3]) * float(binType1[3:5]) * float(binType1[5:7]))
            remainingBinVolume = float(totalBinOfType - filledAmtOfBin) * binVolume  # Check for Remaining Vol in Bin 
            partsAllocated = min(math.floor(remainingBinVolume / partVolume), partOHInven)

            if partsAllocated > 0:
                binFound = True
                break
            #  OLD Code
            # if (df_binData[df_binData['Bin Label'] == binType1]['Availiability Flag'].values[0] == 'Yes'):
            #     binData = df_binData[df_binData['Bin Label'] == binType1]
            #     fillAmt = df_binData[df_binData['Bin Label'] == binType1]['Filled Amount'].values[0] 
            #     totalBin = df_binData[df_binData['Bin Label'] == binType1]['Total Bins'].values[0] 
            #     if (fillAmt < (totalBin - 0.01)):
            #         break

        if not (binFound):
            df_Main.loc[df_Main['Part#'] == pn, 'Actual Bin Allocation'] = "Part2- No More Available/Fitting Bins"
            df_Main.loc[df_Main['Part#'] == pn, 'Overflow Bins'] = "Part2- No More Available/Fitting Bins"
            df_Main.loc[df_Main['Part#'] == pn, 'Overflow Comment'] = f"{partOHInven} Parts Left; No Bins availiable to fit"
            continue

        binType = binData['Bin Label'].values[0]
        totalBinOfType = binData['Total Bins'].values[0]
        filledAmtOfBin = binData['Filled Amount'].values[0]
        binOrder = binData['Bin Order'].values[0]
        binVolume = fillFactor * (float(binType[1:3]) * float(binType[3:5]) * float(binType[5:7]))
        remainingBinVolume = float(totalBinOfType - filledAmtOfBin) * binVolume   ## Check for Remaining Vol in Bin 
        partVolume = partHeight * partWidth * partDepth 
        totalPartVolume = partOHInven * partVolume
        numBins = round(totalPartVolume / binVolume, 3)         ### Number of Bins Required to fill Inventry Parts
        partsAllocated = min(math.floor(remainingBinVolume / partVolume), partOHInven)
        actualBinRequired = round((partsAllocated * partVolume) / binVolume, 3)
        
        df_binData.loc[df_binData['Bin Label'] == binType, 'Filled Amount'] += actualBinRequired
        actualBin = f"{binType} ({actualBinRequired}, {partsAllocated})"

        if filledAmtOfBin + actualBinRequired >= (totalBinOfType - 0.01): 
            df_binData.loc[df_binData['Bin Label'] == binType, 'Availiability Flag'] = 'No'

        if filledAmtOfBin + numBins > totalBinOfType:   # * If Overflow (Actual Bin can't fit all Parts) (Only 1)
            overflowParts = partOHInven - partsAllocated
            for binType1 in df_binData.loc[(df_binData['Bin Order'] > binOrder), 'Bin Label']:
                # Calculate Variables
                binData = df_binData[df_binData['Bin Label'] == binType1]
                totalBinOfType = binData['Total Bins'].values[0]
                filledAmtOfBin = binData['Filled Amount'].values[0]
                binVolume = fillFactor * (float(binType1[1:3]) * float(binType1[3:5]) * float(binType1[5:7]))
                remainingBinVolume = float(totalBinOfType - filledAmtOfBin) * binVolume  # Check for Remaining Vol in Bin 
                partsAllocated = min(math.floor(remainingBinVolume / partVolume), partOHInven)

                if partsAllocated > 0:
                    binFound = True
                    break
                #  OLD Code
                # if (df_binData[df_binData['Bin Label'] == binType1]['Availiability Flag'].values[0] == 'Yes'):
                #     binData = df_binData[df_binData['Bin Label'] == binType1]
                #     fillAmt = df_binData[df_binData['Bin Label'] == binType1]['Filled Amount'].values[0] 
                #     totalBin = df_binData[df_binData['Bin Label'] == binType1]['Total Bins'].values[0] 
                #     if (fillAmt < (totalBin - 0.01)):
                #         break

            if not (binFound):
                df_Main.loc[df_Main['Part#'] == pn, 'Overflow Bins'] = f"Part3- No More Available/Fitting Bins"
                df_Main.loc[df_Main['Part#'] == pn, 'Overflow Comment'] = f"{overflowParts} Parts Left; No Bins availiable to fit"
                continue

            binType = binData['Bin Label'].values[0]
            binVolume = fillFactor * (float(binType[1:3]) * float(binType[3:5]) * float(binType[5:7]))
            totalBinOfType = binData['Total Bins'].values[0]
            filledAmtOfBin = binData['Filled Amount'].values[0]
            remainingBinVolume = float(totalBinOfType - filledAmtOfBin) * binVolume  # Check for Remaining Vol in Bin 
            partsAllocated = min(math.floor(remainingBinVolume / partVolume), overflowParts)
            actualBinRequired = round((partsAllocated * partVolume) / binVolume, 3)
            
            df_binData.loc[df_binData['Bin Label'] == binType, 'Filled Amount'] += actualBinRequired
            overflowBin = f"{binType} ({actualBinRequired}, {partsAllocated})"
        
            leftParts = overflowParts - partsAllocated
            if leftParts > 0:
                binsNeeded = round(leftParts * partVolume / binVolume, 2)
                overflowComment = f"Second Overflow: {leftParts} Parts Left; {binsNeeded} quantity of {binType} Bin Needed;"

            if filledAmtOfBin + actualBinRequired >= (totalBinOfType - 0.01): 
                df_binData.loc[df_binData['Bin Label'] == binType, 'Availiability Flag'] = 'No'

    df_Main.loc[df_Main['Part#'] == pn, 'Actual Bin Allocation'] = actualBin
    df_Main.loc[df_Main['Part#'] == pn, 'Overflow Bins'] = overflowBin
    df_Main.loc[df_Main['Part#'] == pn, 'Overflow Comment'] = overflowComment

# ~1 min 15 secs

In [ ]:

## @ Green and Blue Zone Non-Specialty Parts
for pn in tqdm(df_Main.loc[(df_Main['Zone'] == 'Green') | (df_Main['Zone'] == 'Blue'), 'Part#'], "Completion"):
    # Get Variable
    actualBin =  ""
    overflowBin = ""
    overflowComment = ""

    partData = df_Main[df_Main['Part#'] == pn]
    partHeight = partData['Height'].values[0]
    partWidth = partData['Width'].values[0]
    partDepth = partData['Depth'].values[0]
    binType = partData['Bin Type'].values[0]
    storageType = partData['StorageType'].values[0]
    partOHInven = partData['OH Inventory'].values[0]

    ## * If Inventry Qty is zero, Go to Next Part -- Handle Specialty Storage separately 
    if (binType == "") or ("Specialty Storage" in storageType) or (partOHInven <= 0):     
        continue

    binData = df_binData.loc[df_binData['Bin Label'] == binType]
    totalBinOfType = binData['Total Bins'].values[0]
    filledAmtOfBin = binData['Filled Amount'].values[0]
    binOrder = binData['Bin Order'].values[0]
    partVolume = partHeight * partWidth * partDepth
 
    # Calculate Variables
    binVolume = fillFactor * (float(binType[1:3]) * float(binType[3:5]) * float(binType[5:7]))
    remainingBinVolume = float(totalBinOfType - filledAmtOfBin) * binVolume  # Check for Remaining Vol in Bin 
    partsAllocated = min(math.floor(remainingBinVolume / partVolume), partOHInven)

    if partsAllocated > 0: # & (binData['Availiability Flag'].values[0] == "Yes"):    # If Actual Bin is Availiable
        binVolume = fillFactor * (float(binType[1:3]) * float(binType[3:5]) * float(binType[5:7]))
        remainingBinVolume = float(totalBinOfType - filledAmtOfBin) * binVolume  # Check for Remaining Vol in Bin 
        partVolume = partHeight * partWidth * partDepth 
        totalPartVolume = partOHInven * partVolume
        numBins = round(totalPartVolume / binVolume, 3)
        partsAllocated = min(math.floor(remainingBinVolume / partVolume), partOHInven)
        binRequired = round((partsAllocated * partVolume) / binVolume, 3)    

        df_binData.loc[df_binData['Bin Label'] == binType, 'Filled Amount'] += binRequired
        actualBin = f"{binType} ({binRequired}, {partsAllocated})"

        if filledAmtOfBin + ((partsAllocated * partVolume) / binVolume) >= (totalBinOfType - 0.01):    ### @ ACtual BIN
            df_binData.loc[df_binData['Bin Label'] == binType, 'Availiability Flag'] = 'No'

        ###  Handle OVERFLOW  condition (Only 1 Overflow) 
        if filledAmtOfBin + numBins > totalBinOfType:
            
            overflowParts = partOHInven - partsAllocated
            for binType1 in df_binData.loc[(df_binData['Bin Order'] > binOrder), 'Bin Label']:
                # Calculate Variables
                binData = df_binData[df_binData['Bin Label'] == binType1]
                totalBinOfType = binData['Total Bins'].values[0]
                filledAmtOfBin = binData['Filled Amount'].values[0]
                binVolume = fillFactor * (float(binType1[1:3]) * float(binType1[3:5]) * float(binType1[5:7]))
                remainingBinVolume = float(totalBinOfType - filledAmtOfBin) * binVolume  # Check for Remaining Vol in Bin 
                partsAllocated = min(math.floor(remainingBinVolume / partVolume), partOHInven)

                if partsAllocated > 0:   # @ If Actual Bin is Availiable
                    binFound = True
                    break
                #  OLD Code
                # if (df_binData[df_binData['Bin Label'] == binType1]['Availiability Flag'].values[0] == 'Yes'):
                #     binData = df_binData[df_binData['Bin Label'] == binType1]
                #     fillAmt = df_binData[df_binData['Bin Label'] == binType1]['Filled Amount'].values[0] 
                #     totalBin = df_binData[df_binData['Bin Label'] == binType1]['Total Bins'].values[0] 
                #     if (fillAmt < (totalBin - 0.01)):
                #         break

            if not (binFound):
                df_Main.loc[df_Main['Part#'] == pn, 'Overflow Bins'] = f"Part1-No More Available/Fitting Bins"
                df_Main.loc[df_Main['Part#'] == pn, 'Overflow Comment'] = f"{overflowParts} Parts Left; No Bins availiable to fit"
                continue

            binType = binData['Bin Label'].values[0]
            binVolume = fillFactor * (float(binType[1:3]) * float(binType[3:5]) * float(binType[5:7]))
            totalBinOfType = binData['Total Bins'].values[0]
            filledAmtOfBin = binData['Filled Amount'].values[0]
            remainingBinVolume = float(totalBinOfType - filledAmtOfBin) * binVolume  # Check for Remaining Vol in Bin 
            partsAllocated = min(math.floor(remainingBinVolume / partVolume), overflowParts)
            binRequired = round((partsAllocated * partVolume) / binVolume, 3)  

            overflowBin = f"{binType} ({binRequired}, {partsAllocated})"
        
            leftParts = overflowParts - partsAllocated
            if leftParts > 0:
                binsNeeded = round(leftParts * partVolume / binVolume, 3)
                overflowComment = f"Second Overflow: {leftParts} Parts Left; {binsNeeded} quantity of {binType} Bin Needed;"
            
            if (filledAmtOfBin + binRequired) >= (totalBinOfType - 0.01): 
                df_binData.loc[df_binData['Bin Label'] == binType, 'Availiability Flag'] = 'No'

    else:    ## ^  If  suggested Bin Is NOT Avaialble.  Pick Next Available Bin and Process
        for binType1 in df_binData.loc[(df_binData['Bin Order'] > binOrder), 'Bin Label']:
            # Calculate Variables
            binData = df_binData[df_binData['Bin Label'] == binType1]
            totalBinOfType = binData['Total Bins'].values[0]
            filledAmtOfBin = binData['Filled Amount'].values[0]
            binVolume = fillFactor * (float(binType1[1:3]) * float(binType1[3:5]) * float(binType1[5:7]))
            remainingBinVolume = float(totalBinOfType - filledAmtOfBin) * binVolume  # Check for Remaining Vol in Bin 
            partsAllocated = min(math.floor(remainingBinVolume / partVolume), partOHInven)

            if partsAllocated > 0:   # @ If Actual Bin is Availiable
                binFound = True
                break
            #  OLD Code
            # if (df_binData[df_binData['Bin Label'] == binType1]['Availiability Flag'].values[0] == 'Yes'):
            #     binData = df_binData[df_binData['Bin Label'] == binType1]
            #     fillAmt = df_binData[df_binData['Bin Label'] == binType1]['Filled Amount'].values[0] 
            #     totalBin = df_binData[df_binData['Bin Label'] == binType1]['Total Bins'].values[0] 
            #     if (fillAmt < (totalBin - 0.01)):
            #         break

        if not (binFound):            
            df_Main.loc[df_Main['Part#'] == pn, 'Actual Bin Allocation'] = "Part2-No More Available/Fitting Bins"
            df_Main.loc[df_Main['Part#'] == pn, 'Overflow Bins'] = "Part2-No More Available/Fitting Bins"
            df_Main.loc[df_Main['Part#'] == pn, 'Overflow Comment'] = f"{partOHInven} Parts Left; No Bins availiable to fit"
            continue

        binType = binData['Bin Label'].values[0]
        totalBinOfType = binData['Total Bins'].values[0]
        filledAmtOfBin = binData['Filled Amount'].values[0]
        binOrder = binData['Bin Order'].values[0]
        binVolume = fillFactor * (float(binType[1:3]) * float(binType[3:5]) * float(binType[5:7]))
        remainingBinVolume = float(totalBinOfType - filledAmtOfBin) * binVolume   ## Check for Remaining Vol in Bin 
        partVolume = partHeight * partWidth * partDepth 
        totalPartVolume = partOHInven * partVolume
        numBins = round(totalPartVolume / binVolume, 3)         ### Number of Bins Required to fill Inventry Parts
        partsAllocated = min(math.floor(remainingBinVolume / partVolume), partOHInven)
        binRequired = round((partsAllocated * partVolume) / binVolume, 3)  
        
        df_binData.loc[df_binData['Bin Label'] == binType, 'Filled Amount'] += binRequired
        actualBin = f"{binType} ({binRequired}, {partsAllocated})"

        if (filledAmtOfBin + binRequired) >= (totalBinOfType - 0.01):
            df_binData.loc[df_binData['Bin Label'] == binType, 'Availiability Flag'] = 'No'

        ###  Handle OVERFLOW  condition (Only 1 Overflow) 
        if filledAmtOfBin + numBins > totalBinOfType:
            overflowParts = partOHInven - partsAllocated
            for binType1 in df_binData.loc[(df_binData['Bin Order'] > binOrder), 'Bin Label']:
                # Calculate Variables
                binData = df_binData[df_binData['Bin Label'] == binType1]
                totalBinOfType = binData['Total Bins'].values[0]
                filledAmtOfBin = binData['Filled Amount'].values[0]
                binVolume = fillFactor * (float(binType1[1:3]) * float(binType1[3:5]) * float(binType1[5:7]))
                remainingBinVolume = float(totalBinOfType - filledAmtOfBin) * binVolume  # Check for Remaining Vol in Bin 
                partsAllocated = min(math.floor(remainingBinVolume / partVolume), partOHInven)

                if partsAllocated > 0:   # @ If Actual Bin is Availiable
                    binFound = True
                    break
                #  OLD Code
                # if (df_binData[df_binData['Bin Label'] == binType1]['Availiability Flag'].values[0] == 'Yes'):
                #     binData = df_binData[df_binData['Bin Label'] == binType1]
                #     fillAmt = df_binData[df_binData['Bin Label'] == binType1]['Filled Amount'].values[0] 
                #     totalBin = df_binData[df_binData['Bin Label'] == binType1]['Total Bins'].values[0] 
                #     if (fillAmt < (totalBin - 0.01)):
                #         break

            if not (binFound):
                df_Main.loc[df_Main['Part#'] == pn, 'Overflow Bins'] = f"Part3-No More Available/Fitting Bins"
                df_Main.loc[df_Main['Part#'] == pn, 'Overflow Comment'] = f"{overflowParts} Parts Left; No Bins availiable to fit"
                continue

            binType = binData['Bin Label'].values[0]
            binVolume = fillFactor * (float(binType[1:3]) * float(binType[3:5]) * float(binType[5:7]))
            totalBinOfType = binData['Total Bins'].values[0]
            filledAmtOfBin = binData['Filled Amount'].values[0]
            remainingBinVolume = float(totalBinOfType - filledAmtOfBin) * binVolume  # Check for Remaining Vol in Bin 
            partsAllocated = min(math.floor(remainingBinVolume / partVolume), overflowParts)
            binRequired = round((partsAllocated * partVolume) / binVolume, 3) 

            ## @ ADD Below Filled Amount for Overflow bins
            df_binData.loc[df_binData['Bin Label'] == binType, 'Filled Amount'] += binRequired
            overflowBin = f"{binType} ({binRequired}, {partsAllocated})"
        
            leftParts = overflowParts - partsAllocated
            if leftParts > 0:
                binsNeeded = round(leftParts * partVolume / binVolume, 3)
                overflowComment = f"Second Overflow: {leftParts} Parts Left; {binsNeeded} quantity of {binType} Bin Needed;"

            if (filledAmtOfBin + binRequired) >= (totalBinOfType - 0.01):
                df_binData.loc[df_binData['Bin Label'] == binType, 'Availiability Flag'] = 'No'

    df_Main.loc[df_Main['Part#'] == pn, 'Actual Bin Allocation'] = actualBin
    df_Main.loc[df_Main['Part#'] == pn, 'Overflow Bins'] = overflowBin
    df_Main.loc[df_Main['Part#'] == pn, 'Overflow Comment'] = overflowComment

    
# ~16 mins 12 secs

In [ ]:

## @ All Zone Specialty Parts
for pn in df_Main['Part#']:
    # Get & Set Variables
    actualBin =  ""
    overflowBin = ""
    overflowComment = ""

    partData = df_Main[df_Main['Part#'] == pn]
    partHeight = partData['Height'].values[0]
    partWidth = partData['Width'].values[0]
    partDepth = partData['Depth'].values[0]
    binType = partData['Bin Type'].values[0]
    partOHInven = partData['OH Inventory'].values[0]
    partSKUCount = partData['SKU Count'].values[0]

    # ~ Base Continue Case - If no Storage Assignment, no Inventory Parts or is not Specialty Part
    if (binType == "") or (partOHInven <= 0):     
        continue
    if (all([binLabelTypes not in binType.lower() for binLabelTypes in ['br', 'tr', 'hs', 'bc', 'bh']])):     
        continue

    binData = df_binData.loc[df_binData['Bin Label'] == binType]
    totalBinOfType = binData['Total Bins'].values[0]
    filledAmtOfBin = binData['Filled Amount'].values[0]

    ## * Calculation for Battery
    if "br" in binType.lower():
        remainingBinWidth = (totalBinOfType - filledAmtOfBin) * float(binType[4:6])
        partsAllocated = min(math.floor(remainingBinWidth / partWidth), partOHInven)
        binsRequired = round((partsAllocated * partWidth) / float(binType[4:6]), 4)
        partsLeft = partOHInven - partsAllocated
        overflowBinsNeeded = partsLeft / float(binType[2:4])
    ## * Calculation for Tire
    if "tr" in binType.lower():
        remainingBinWidth = (totalBinOfType - filledAmtOfBin) * float(binType[2:4])
        partsAllocated = min(math.floor(remainingBinWidth / partWidth), partOHInven)
        binsRequired = round((partsAllocated * partWidth) / float(binType[2:4]), 4)
        partsLeft = partOHInven - partsAllocated
        overflowBinsNeeded = round(partsLeft / float(binType[2:4]), 3)
    ## * Calculation for Hanging Storage
    if "hs" in binType.lower():
        partsAllocated = min(math.floor(totalBinOfType - filledAmtOfBin), partOHInven)
        binsRequired = round(partsAllocated, 4)
        partsLeft = partOHInven - partsAllocated
        overflowBinsNeeded = partsLeft
    ## * Calculation for Bumper Cover & Hoods
    if ("bc" in binType.lower()) | ("bh" in binType.lower()):
        partVolume = partHeight * partWidth * partDepth 
        binVol = fillFactor * (float(binType[2:4]) * float(binType[4:6]) * float(binType[6:8]))
        partsAllocated = min(math.floor(((totalBinOfType - filledAmtOfBin) * binVol) / partVolume), partOHInven)
        binsRequired = round((partsAllocated * partVolume) / binVol, 4)
        partsLeft = partOHInven - partsAllocated
        overflowBinsNeeded = round((partsLeft * partVolume) / binVolume, 3)

    ## * Main Data Update
    if filledAmtOfBin + binsRequired >= (totalBinOfType - 0.01): # Update Availiability Flag, if full
        df_binData.loc[df_binData['Bin Label'] == binType, 'Availiability Flag'] = 'No'

    if partsAllocated > 0: # If any parts allocated
        df_binData.loc[df_binData['Bin Label'] == binType, 'Filled Amount'] += binsRequired
        actualBin = f"{binType} ({binsRequired}, {partsAllocated})"
    else:
        actualBin = "SP- No More Available/Fitting Bins" # If no parts can be allocated

    if partsLeft > 0: # If Overflow
        overflowBin = "SP- No More Available/Fitting Bins"
        overflowComment = f"{partOHInven - partsAllocated} Parts Left; {overflowBinsNeeded} quantity of {binType} Bin Needed;"

    df_Main.loc[df_Main['Part#'] == pn, 'Actual Bin Allocation'] = actualBin
    df_Main.loc[df_Main['Part#'] == pn, 'Overflow Bins'] = overflowBin
    df_Main.loc[df_Main['Part#'] == pn, 'Overflow Comment'] = overflowComment
           

# ~ 11 mins 28 sec

In [ ]:
df_Main

In [ ]:

# * Write  the Final Dataset into Excel
df_Main.to_excel('Final_Dataset.xlsx', index=False) 
# * Write  the Final BINS Dataset into Excel
df_binData.to_excel('All_Bins_Data.xlsx', index=False)

In [ ]:
wb = openpyxl.load_workbook('Final_Dataset.xlsx')
ws = wb.active

# Adjust column widths
ws.column_dimensions['A'].width = 16
ws.column_dimensions['B'].width = 52
ws.column_dimensions['C'].width = 13
ws.column_dimensions['D'].width = 0
ws.column_dimensions['E'].width = 7
ws.column_dimensions['F'].width = 15
ws.column_dimensions['G'].width = 12
ws.column_dimensions['H'].width = 12
ws.column_dimensions['I'].width = 13
ws.column_dimensions['J'].width = 11
ws.column_dimensions['K'].width = 12
ws.column_dimensions['L'].width = 9
ws.column_dimensions['M'].width = 9
ws.column_dimensions['N'].width = 9
ws.column_dimensions['O'].width = 10
ws.column_dimensions['P'].width = 24
ws.column_dimensions['Q'].width = 34
ws.column_dimensions['R'].width = 12
ws.column_dimensions['S'].width = 18
ws.column_dimensions['T'].width = 33
ws.column_dimensions['U'].width = 32
ws.column_dimensions['V'].width = 46
ws.column_dimensions['W'].width = 12

ws.column_dimensions['D'].hidden = True

ws.freeze_panes = 'A2'

for cell in ws['B']:
    cell.alignment = openpyxl.styles.Alignment(horizontal='center', vertical='center')
for cell in ws['E']:
    cell.alignment = openpyxl.styles.Alignment(horizontal='center', vertical='center')
for cell in ws['O']:
    cell.alignment = openpyxl.styles.Alignment(horizontal='center', vertical='center')
for cell in ws['P']:
    cell.alignment = openpyxl.styles.Alignment(horizontal='center', vertical='center')
for cell in ws['Q']:
    cell.alignment = openpyxl.styles.Alignment(horizontal='center', vertical='center')
for cell in ws['R']:
    cell.alignment = openpyxl.styles.Alignment(horizontal='center', vertical='center')
for cell in ws['S']:
    cell.alignment = openpyxl.styles.Alignment(horizontal='center', vertical='center')
for cell in ws['T']:
    cell.alignment = openpyxl.styles.Alignment(horizontal='center', vertical='center')
for cell in ws['U']:
    cell.alignment = openpyxl.styles.Alignment(horizontal='center', vertical='center')
for cell in ws['V']:
    cell.alignment = openpyxl.styles.Alignment(horizontal='center', vertical='center')


wb.save('Final_Dataset.xlsx')

In [ ]:
# ## * Main Loop for All Parts 
# for pn in df_Main['Part#']:
#     # Get & Set Variables
#     actualBin =  ""
#     overflowBin = ""
#     overflowComment = ""

#     partData = df_Main[df_Main['Part#'] == pn]
#     partHeight = partData['Height'].values[0]
#     partWidth = partData['Width'].values[0]
#     partDepth = partData['Depth'].values[0]
#     partVolume = partHeight * partWidth * partDepth 
#     binType = partData['Bin Type'].values[0]
#     partOHInven = partData['OH Inventory'].values[0]
#     totalPartVolume = partOHInven * partVolume

#     # ~ Base Continue Case - If no Storage Assignment, no Inventory Parts or is Specialty Part
#     if (binType == "") or (any([binLabelTypes in binType.lower() for binLabelTypes in ['br', 'tr', 'hs', 'bc', 'bh']])) or (partOHInven == 0):     
#         continue

#     binData = df_binData.loc[df_binData['Bin Label'] == binType]
#     totalBinOfType = binData['Total Bins'].values[0]
#     filledAmtOfBin = binData['Filled Amount'].values[0]
#     binOrder = binData['Bin Order'].values[0]

#     remainingBinVolume = float(totalBinOfType - filledAmtOfBin) * binVolume  # Check for Remaining Vol in Bin 
#     partsAllocated = min(math.floor(remainingBinVolume / partVolume), partOHInven)

#     if partsAllocated > 0:   # * If Actual Bin is Availiable
#         # Calculate Variables
#         binVolume = fillFactor * (float(binType[1:3]) * float(binType[3:5]) * float(binType[5:7]))
#         actualBinRequired = round((partsAllocated * partVolume) / binVolume, 3)
#         numBins = round(totalPartVolume / binVolume, 3)
        
#         # Set Values
#         df_binData.loc[df_binData['Bin Label'] == binType, 'Filled Amount'] += actualBinRequired
#         actualBin = f"{binType} ({actualBinRequired}, {partsAllocated})"

#         if filledAmtOfBin + actualBinRequired >= (totalBinOfType - 0.01): # Update Availiability Flag, if full
#             df_binData.loc[df_binData['Bin Label'] == binType, 'Availiability Flag'] = 'No'

#         if filledAmtOfBin + numBins > totalBinOfType:   # * If Overflow (Actual Bin can't fit all Parts) (Only 1)
#             overflowParts = partOHInven - partsAllocated
            
#             # Find next suitable availiable bin to fill overflow parts
#             for newBinType in df_binData.loc[(df_binData['Bin Order'] > binOrder), 'Bin Label']:   
#                 if (df_binData[df_binData['Bin Label'] == newBinType]['Availiability Flag'].values[0] == 'Yes'):
#                     binData = df_binData[df_binData['Bin Label'] == newBinType]
#                     break

#             if (binType == binData['Bin Label'].values[0]): # If no bins found
#                 df_Main.loc[df_Main['Part#'] == pn, 'Overflow Bins'] = f"Part1 - No More Available/Fitting Bins"
#                 df_Main.loc[df_Main['Part#'] == pn, 'Overflow Comment'] = f"{overflowParts} Parts Left; No Bins availiable to fit"
#                 continue

#             # Get & Calculate Variables
#             binType = binData['Bin Label'].values[0]
#             totalBinOfType = binData['Total Bins'].values[0]
#             filledAmtOfBin = binData['Filled Amount'].values[0]
            
#             binVolume = fillFactor * (float(binType[1:3]) * float(binType[3:5]) * float(binType[5:7]))
#             remainingBinVolume = float(totalBinOfType - filledAmtOfBin) * binVolume  # Check for Remaining Vol in Bin 
#             partsAllocated = min(math.floor(remainingBinVolume / partVolume), overflowParts)
#             actualBinRequired = round((partsAllocated * partVolume) / binVolume, 3)

#             # Set Values -- Add Filled Amount & Build overflowBin string
#             df_binData.loc[df_binData['Bin Label'] == binType, 'Filled Amount'] += actualBinRequired
#             overflowBin = f"{binType} ({actualBinRequired}, {partsAllocated})"
        
#             # If more overflow
#             leftParts = overflowParts - partsAllocated
#             if leftParts > 0:
#                 binsNeeded = round(leftParts * partVolume / binVolume, 3)
#                 overflowComment = f"More Overflow: {leftParts} Parts Left; {binsNeeded} quantity of {binType} Bin Needed;"
            
#             if filledAmtOfBin + actualBinRequired >= (totalBinOfType - 0.01): # Update Availiability Flag, if full
#                 df_binData.loc[df_binData['Bin Label'] == binType, 'Availiability Flag'] = 'No'

#     else:   # * If Main Bin not Availiable - Pick next suitable availiable Bin
#         for newBinType in df_binData.loc[(df_binData['Bin Order'] > binOrder), 'Bin Label']:
#             if (df_binData[df_binData['Bin Label'] == newBinType]['Availiability Flag'].values[0] == 'Yes'):
#                 binData = df_binData[df_binData['Bin Label'] == newBinType]
#                 #  & (df_binData[df_binData['Bin Label'] == binType]["Bin Label"][1:3].isdigit())
#                 break
                
#         if (binType == binData['Bin Label'].values[0]):
#             df_Main.loc[df_Main['Part#'] == pn, 'Actual Bin Allocation'] = "Part2 - No More Available/Fitting Bins"
#             df_Main.loc[df_Main['Part#'] == pn, 'Overflow Bins'] = "No More Available/Fitting Bins"
#             df_Main.loc[df_Main['Part#'] == pn, 'Overflow Comment'] = f"{partOHInven} Parts Left; No Bins availiable to fit"
#             continue

#         binType = binData['Bin Label'].values[0]
#         totalBinOfType = binData['Total Bins'].values[0]
#         filledAmtOfBin = binData['Filled Amount'].values[0]
#         binOrder = binData['Bin Order'].values[0]
#         binVolume = fillFactor * (float(binType[1:3]) * float(binType[3:5]) * float(binType[5:7]))
#         remainingBinVolume = float(totalBinOfType - filledAmtOfBin) * binVolume   ## Check for Remaining Vol in Bin 
#         partVolume = partHeight * partWidth * partDepth 
#         totalPartVolume = partOHInven * partVolume
#         numBins = round(totalPartVolume / binVolume, 2)
#         partsAllocated = min(math.floor(remainingBinVolume / partVolume), partOHInven)
#         actualBinRequired = round((partsAllocated * partVolume) / binVolume, 3)
        
#         df_binData.loc[df_binData['Bin Label'] == binType, 'Filled Amount'] += actualBinRequired
#         actualBin = f"{binType} ({actualBinRequired}, {partsAllocated})"

#         if filledAmtOfBin + actualBinRequired >= (totalBinOfType - 0.01): # Update Availiability Flag, if full
#             df_binData.loc[df_binData['Bin Label'] == binType, 'Availiability Flag'] = 'No'

#         # * If Overflow (Actual Bin can't fit all Parts) (Only 1)
#         if filledAmtOfBin + numBins > totalBinOfType:
#             overflowParts = partOHInven - partsAllocated
#             for binType in df_binData.loc[(df_binData['Bin Order'] > binOrder), 'Bin Label']:
#                 if (df_binData[df_binData['Bin Label'] == binType]['Availiability Flag'].values[0] == 'Yes'):
#                     binData = df_binData[df_binData['Bin Label'] == binType]
#                     break

#             if (binType == binData['Bin Label'].values[0]):
#                 df_Main.loc[df_Main['Part#'] == pn, 'Overflow Bins'] = f"Part3- No More Available/Fitting Bins"
#                 df_Main.loc[df_Main['Part#'] == pn, 'Overflow Comment'] = f"{overflowParts} Parts Left; No Bins availiable to fit"
#                 continue

#             binType = binData['Bin Label'].values[0]
#             binVolume = fillFactor * (float(binType[1:3]) * float(binType[3:5]) * float(binType[5:7]))
#             totalBinOfType = binData['Total Bins'].values[0]
#             filledAmtOfBin = binData['Filled Amount'].values[0]
#             remainingBinVolume = float(totalBinOfType - filledAmtOfBin) * binVolume  # Check for Remaining Vol in Bin 
#             partsAllocated = min(math.floor(remainingBinVolume / partVolume), overflowParts)
#             actualBinRequired = round((partsAllocated * partVolume) / binVolume, 3)
            
#             # Set Values -- Add Filled Amount & Build overflowBin string
#             df_binData.loc[df_binData['Bin Label'] == binType, 'Filled Amount'] += actualBinRequired
#             overflowBin = f"{binType} ({actualBinRequired}, {partsAllocated})"
        
#             leftParts = overflowParts - partsAllocated
#             if leftParts > 0:
#                 binsNeeded = round(leftParts * partVolume / binVolume, 2)
#                 overflowComment = f"Second Overflow: {leftParts} Parts Left; {binsNeeded} quantity of {binType} Bin Needed;"

#             if filledAmtOfBin + actualBinRequired >= (totalBinOfType - 0.01): 
#                 df_binData.loc[df_binData['Bin Label'] == binType, 'Availiability Flag'] = 'No'

#     df_Main.loc[df_Main['Part#'] == pn, 'Actual Bin Allocation'] = actualBin
#     df_Main.loc[df_Main['Part#'] == pn, 'Overflow Bins'] = overflowBin
#     df_Main.loc[df_Main['Part#'] == pn, 'Overflow Comment'] = overflowComment

# ~ 20 sec